### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [4]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [5]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [6]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [7]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


Error processing line 1 of /home/krish/Downloads/Codes/AI/churn model/venv/lib/python3.11/site-packages/distutils-precedence.pth:

Error processing line 1 of /home/krish/Downloads/Codes/AI/churn model/venv/lib/python3.11/site-packages/distutils-precedence.pth:

Error processing line 1 of /home/krish/Downloads/Codes/AI/churn model/venv/lib/python3.11/site-packages/distutils-precedence.pth:

Error processing line 1 of /home/krish/Downloads/Codes/AI/churn model/venv/lib/python3.11/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 186, in addpackage
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored
Error processing line 1 of /home/krish/Downloads/Codes/AI/churn model/venv/lib/python3.11/site-packages/distutils-precedence.pth:

Error processing line 1 of /home/krish/Downloads/Codes/AI/churn model/venv/lib/python3.11/site-packages/distutils-precedence.pth:

Er

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
167/167 [==============================] - 1s 3ms/step - loss: 0.6002 - accuracy: 0.6670
Epoch 2/50
167/167 [==============================] - 1s 3ms/step - loss: 0.4960 - accuracy: 0.7715
Epoch 2/50
167/167 [==============================] - 1s 3ms/step - loss: 0.4947 - accuracy: 0.7789
Epoch 2/50
167/167 [==============================] - 2s 4ms/step - loss: 0.5848 - accuracy: 0.7135
Epoch 3/50
127/167 [=====================>........] - ETA: 0s - loss: 0.4840 - accuracy: 0.7857Epoch 3/50
Epoch 3/50
167/167 [==============================] - 0s 3ms/step - loss: 0.4133 - accuracy: 0.8254
Epoch 3/50
167/167 [==============================] - 1s 3ms/step - loss: 0.4494 - accuracy: 0.7999
Epoch 3/50
167/167 [==============================] - 1s 4ms/step - loss: 0.4521 - accuracy: 0.8065
Epoch 3/50
167/167 [=========

/home/krish/Downloads/Codes/AI/churn model/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/krish/Downloads/Codes/AI/churn model/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


167/167 [==============================] - 0s 3ms/step - loss: 0.3155 - accuracy: 0.8706
Epoch 50/50
167/167 [==============================] - 0s 3ms/step - loss: 0.2933 - accuracy: 0.8753
Epoch 48/50
167/167 [==============================] - 0s 3ms/step - loss: 0.3208 - accuracy: 0.8682
Epoch 50/50
167/167 [==============================] - 0s 3ms/step - loss: 0.3177 - accuracy: 0.8667
Epoch 48/50
167/167 [==============================] - 0s 3ms/step - loss: 0.3127 - accuracy: 0.8701
Epoch 49/50
167/167 [==============================] - 0s 3ms/step - loss: 0.3222 - accuracy: 0.8674
Epoch 50/50
  1/167 [..............................] - ETA: 0s - loss: 0.2636 - accuracy: 0.8750

...

Epoch 32/100
167/167 [==============================] - 1s 3ms/step - loss: 0.3109 - accuracy: 0.8714
Epoch 30/100
167/167 [==============================] - 0s 3ms/step - loss: 0.3083 - accuracy: 0.8706
Epoch 72/100
167/167 [==============================] - 1s 3ms/step - loss: 0.2867 - accuracy: 0.8800
Epoch 29/100
167/167 [==============================] - 0s 3ms/step - loss: 0.3295 - accuracy: 0.8642
Epoch 32/100
167/167 [==============================] - 1s 4ms/step - loss: 0.2920 - accuracy: 0.8772
Epoch 82/100
167/167 [==============================] - 1s 4ms/step - loss: 0.3061 - accuracy: 0.8753
Epoch 82/100
167/167 [==============================] - 0s 3ms/step - loss: 0.2752 - accuracy: 0.8847
Epoch 30/100
167/167 [==============================] - 0s 3ms/step - loss: 0.2786 - accuracy: 0.8800
Epoch 31/100
167/167 [==============================] - 0s 3ms/step - loss: 0.3294 - accuracy: 0.8644
Epoch 33/100
167/167 [==============================] - 1s 3ms/step - loss: 0.2863

/home/krish/Downloads/Codes/AI/churn model/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


167/167 [==============================] - 0s 3ms/step - loss: 0.1986 - accuracy: 0.9181
Epoch 45/100
167/167 [==============================] - 0s 3ms/step - loss: 0.2478 - accuracy: 0.8911
Epoch 48/100
167/167 [==============================] - 0s 2ms/step - loss: 0.2926 - accuracy: 0.8785
Epoch 51/100
167/167 [==============================] - 0s 2ms/step - loss: 0.2451 - accuracy: 0.8986
Epoch 48/100
167/167 [==============================] - 0s 3ms/step - loss: 0.2916 - accuracy: 0.8815
Epoch 49/100
167/167 [==============================] - 0s 3ms/step - loss: 0.3213 - accuracy: 0.8687
Epoch 51/100
167/167 [==============================] - 0s 3ms/step - loss: 0.2907 - accuracy: 0.8774
Epoch 49/100
167/167 [==============================] - 0s 3ms/step - loss: 0.3052 - accuracy: 0.8716
Epoch 91/100
167/167 [==============================] - 0s 3ms/step - loss: 0.3092 - accuracy: 0.8733
Epoch 52/100
167/167 [==============================] - 0s 2ms/step - loss: 0.2818 - accuracy: 

/home/krish/Downloads/Codes/AI/churn model/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
2025-10-27 10:13:13.246778: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-10-27 10:13:13.249924: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices

Epoch 1/50
250/250 [==============================] - 0s 621us/step - loss: 0.4553 - accuracy: 0.8010
Epoch 2/50
250/250 [==============================] - 0s 533us/step - loss: 0.3919 - accuracy: 0.8369
Epoch 3/50
250/250 [==============================] - 0s 508us/step - loss: 0.3672 - accuracy: 0.8496
Epoch 4/50
250/250 [==============================] - 0s 510us/step - loss: 0.3548 - accuracy: 0.8540
Epoch 5/50
250/250 [==============================] - 0s 509us/step - loss: 0.3497 - accuracy: 0.8561
Epoch 6/50
250/250 [==============================] - 0s 493us/step - loss: 0.3455 - accuracy: 0.8579
Epoch 7/50
250/250 [==============================] - 0s 503us/step - loss: 0.3426 - accuracy: 0.8594
Epoch 8/50
250/250 [==============================] - 0s 523us/step - loss: 0.3413 - accuracy: 0.8604
Epoch 9/50
250/250 [==============================] - 0s 519us/step - loss: 0.3388 - accuracy: 0.8599
Epoch 10/50
250/250 [==============================] - 0s 524us/step - loss: 0.337